<a href="https://colab.research.google.com/github/PrusTalbot/PrusTalbot/blob/main/%D0%9C%D0%98%D0%9D%D0%98-%D0%9A%D0%95%D0%99%D0%A1%20%D0%92%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D0%B5%20%D1%80%D1%8F%D0%B4%D1%8B.%20PROPHET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**МИНИ-КЕЙС
ВРЕМЕННЫЕ РЯДЫ. PROPHET**

**сбор данных, выбор модели, обучение и получение прогноза**

необходимо выполнить следующие действия:
1. прочитать данные в дата фрейм и привести их к нужному типу данных
2. построить графики, оценить визуально ряд. При необходимости, очистить ряд от выбросов
3. проверить ряд на стационарность и выбрать модель построения прогноза
4. сделать тестовый прогноз и оценить ошибку. Принять решение о том, можем ли мы использовать данную модель?
5. сделать финальный прогноз.

In [52]:
import pandas as pd
import numpy as np
import plotly.express as px
from prophet import Prophet
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error, r2_score

In [53]:
from google.colab import files
uploaded=files.upload()

Saving Практическая задача.csv to Практическая задача.csv


In [55]:
df = pd.read_csv('Практическая задача.csv')
df['ds']=pd.to_datetime(df['ds'], yearfirst=True)

In [58]:
df

,ds,y
0,2023-01-01,17.117648
1,2023-01-02,9.960617
2,2023-01-03,11.148019
3,2023-01-04,12.763887
4,2023-01-05,13.595710
...,...,...
360,2023-12-27,28.946305
361,2023-12-28,32.420714
362,2023-12-29,31.176869
363,2023-12-30,33.603544


In [59]:
df.fillna(df.mean(), inplace=True) # Заменяем пропуски средним
df

<ipython-input-59-a8a6b5c7206a>:1: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.



,ds,y
0,2023-01-01,17.117648
1,2023-01-02,9.960617
2,2023-01-03,11.148019
3,2023-01-04,12.763887
4,2023-01-05,13.595710
...,...,...
360,2023-12-27,28.946305
361,2023-12-28,32.420714
362,2023-12-29,31.176869
363,2023-12-30,33.603544


ВИЗУАЛЬНАЯ ОЦЕНКА ВРЕМЕННОГО РЯДА

In [60]:
fig=px.line(df, x='ds', y='y', title='Продажи на каждый день') # Строим крафик зависимости одной переменной от дугой. Если несколько переменных, то y=['n1','n2','...']
fig.show()

*Наблюдаем восходящий тренд. При этом наблюдаются локальные пики и дропы, но важно, чтобы общее напровление движения было на положительные изменения*

*Также, у даннгог ряда наблюдается явная недельная сезонность. Также, во временном ряде наблюдаются небольшие пики и небольшие бропы, которые повторяются каждые 7 дней. Это явно говорит о недельной сезонности*

Однако проверим ряд на стационарность с помощью критерия Дики-Фуллера. Это важный этап работы с данными, так как на основании этого принимается решение о выборе модели

ТЕСТ НА СТАЦИОНАРНОСТЬ ВРЕМЕННОГО РЯДА

In [62]:
time_series=df['y']
result=adfuller(time_series)

print('p-value:', result[1])

if result[1]<=0.05:
  print('Временной ряд стационарен')
else:
  print('Временной ряд НЕстационарен')

p-value: 0.9788434784097652
Временной ряд НЕстационарен


In [64]:
result

(0.33380738018475464,
 0.9788434784097652,
 15,
 349,
 {'1%': -3.449226932880019,
  '5%': -2.869857365438656,
  '10%': -2.571201085130664},
 1168.0398028001741)

In [65]:
index=int(df.shape[0]*0.3)  # Разложили по пропорции 70/30
train, test=df[:-index],df[-index:]

In [66]:
index

109

In [67]:
test

,ds,y
256,2023-09-14,24.057599
257,2023-09-15,26.111434
258,2023-09-16,26.527752
259,2023-09-17,29.255201
260,2023-09-18,20.700564
...,...,...
360,2023-12-27,28.946305
361,2023-12-28,32.420714
362,2023-12-29,31.176869
363,2023-12-30,33.603544


In [68]:
train

,ds,y
0,2023-01-01,17.117648
1,2023-01-02,9.960617
2,2023-01-03,11.148019
3,2023-01-04,12.763887
4,2023-01-05,13.595710
...,...,...
251,2023-09-09,26.561907
252,2023-09-10,28.278152
253,2023-09-11,22.361937
254,2023-09-12,24.034680


ТЕСТОВЫЙ ПРОГОН МОДЕЛИ

In [70]:
model=Prophet()
model.fit(train) # Передаем обучающие данные (train) в метод Prophet

predictions_period=model.make_future_dataframe(periods=index) # Задаем (index) в качестве длительности нашего прогноза
predictions_period=predictions_period[-index:]

int_forecast=model.predict(predictions_period)  # Датафрейм, с данными нашего прогноза
int_forecast[['ds','yhat','yhat_lower','yhat_upper']] # Берем только интересующие нас данные, т.к. массив огромен (дата, значение, нижняя граница, верхняя граница)

mse=mean_squared_error(test['y'], int_forecast['yhat'])#  Считаем mse и передаем в метод реальное значение (test['y')] и пронозное  (int_forecast['yhat'])
r2=r2_score(test['y'], int_forecast['yhat'])

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpv12yrhzf/1x_rvhd5.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpv12yrhzf/szajwlys.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=56967', 'data', 'file=/tmp/tmpv12yrhzf/1x_rvhd5.json', 'init=/tmp/tmpv12yrhzf/szajwlys.json', 'output', 'file=/tmp/tmpv12yrhzf/prophet_modelree_dfa3/prophet_model-20231130142558.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:25:58 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:25:58 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [71]:
r2

0.832194794521548

In [72]:
mse

1.6516313297669012

ФИНАЛЬНЫЙ ПРОГОН МОДЕЛИ. КОНЕЧНЫЙ ДАТАФРЕЙМ

In [73]:
model=Prophet()
model.fit(df)

predictions_period=model.make_future_dataframe(periods=30)

forecast=model.predict(predictions_period)
forecast[['ds','yhat','yhat_lower','yhat_upper']]

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpv12yrhzf/8jr0ix6q.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpv12yrhzf/fc0a6try.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=7660', 'data', 'file=/tmp/tmpv12yrhzf/8jr0ix6q.json', 'init=/tmp/tmpv12yrhzf/fc0a6try.json', 'output', 'file=/tmp/tmpv12yrhzf/prophet_modelems6zrs8/prophet_model-20231130142610.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:26:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:26:10 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


,ds,yhat,yhat_lower,yhat_upper
0,2023-09-14,24.746563,23.501236,25.925360
1,2023-09-15,25.993685,24.701780,27.191161
2,2023-09-16,26.929454,25.719143,28.139688
3,2023-09-17,28.329423,27.089500,29.462558
4,2023-09-18,22.125051,20.854421,23.373057
...,...,...,...,...
104,2023-12-27,29.022093,27.715190,30.256338
105,2023-12-28,29.515171,28.230067,30.858424
106,2023-12-29,30.762293,29.494372,32.077332
107,2023-12-30,31.698062,30.495298,33.063008


Алгоритм действий при решении задач на прогнозирование:
1. Прочитать данные и привести их к нужному типу данных
2. Построить график, оценить визуальный ряд. Очистить от выбросов
3. Проверить ряд на стационарность и выбрать модель построения прогноза
4. Сделать тестовый прогноз и оценить ошибку. Принять решение о том, можем ли мы использовать данную модель
5. Сделать финальный прогноз